In [1]:
import control
from control.matlab import *
import numpy as np
import scipy as sc
import sympy as sy
import matplotlib.pyplot as plt

In [ ]:
def multiply_poly(a):
    p1 = np.array([1])
    for i in range(len(a)):
        polynom = np.array(a[i])
        p1 = sc.signal.convolve(p1, polynom)
    return p1

# Stability

## Routh-Hurwitz

In [73]:
def print_pol(poly, degree):
    print("Polynom:")
    for i in range(degree+1):
        if i == degree:
            print(str(poly[i]),'\n')
        else:
            print(str(poly[i])+"s^"+str(degree-i), end ="+")            

def fill_matrix(m, sz, degree):
    flag = 0
    for i in range(2):
        for j in range(sz):
            if i+2*j > degree:
                break
            m[i][j] = p[i+2*j]
    for i in range(2, degree+1):
        for j in range(sz-1):
            if m[i-1][0] == 0 and flag == 0:
                print("Caso especial uwu")
                m[i-1][0] = 1e-8
                flag = 1
            m[i][j] = (m[i-2][j+1]*m[i-1][0] - m[i-2][0]*m[i-1][j+1])/m[i-1][0]
        if flag == 1:
            flag = 2
            f = True
            for j in range(0, sz-1):
                if(m[i-1][j+1] != 0):
                    f = False
            if f:
                return i-1
    return -1

def special_case(m, pos, sz, degree):
    for i in range(sz):
        m[pos][i] = m[pos-1][i]*(degree-pos+1-2*i)
    for i in range(pos+1, degree+1):
        for j in range(sz-1):
            m[i][j] = (m[i-2][j+1]*m[i-1][0] - m[i-2][0]*m[i-1][j+1])/m[i-1][0]
            
    
def routh_hurwitz(p):
    degree = p.shape[0]-1
    sz = (degree+2)//2
    m = np.zeros((degree+1, sz))

    print_pol(p, degree)
    row = fill_matrix(m, sz, degree)
    if row == -1:
        print("Nice :)")
    else:
        print("Segundo caso especial uwu")
        special_case(m, row, sz, degree)

    print(m)

In [74]:
#p = np.array([1,7,6,42,8,56])
p = np.array([1, 1, 12, 22, 39, 59, 48, 38, 20])

routh_hurwitz(p)

Polynom:
1s^8+1s^7+12s^6+22s^5+39s^4+59s^3+48s^2+38s^1+20 

Caso especial uwu
Segundo caso especial uwu
[[  1.          12.          39.          48.          20.        ]
 [  1.          22.          59.          38.           0.        ]
 [-10.         -20.          10.          20.           0.        ]
 [ 20.          60.          40.           0.           0.        ]
 [ 10.          30.          20.           0.           0.        ]
 [ 40.          60.           0.          -0.          -0.        ]
 [ 15.          20.           0.           0.           0.        ]
 [  6.66666667   0.          -0.          -0.           0.        ]
 [ 20.           0.           0.           0.           0.        ]]


# Steady-State errors

<img src="uwu.png" width=350 height=350 />

$$
E(s) = \frac{R(s)}{1+G(s)}
$$
$$
e(\infty) = \lim_{s\to 0} sE(s) = \lim_{s\to 0} \frac{sR(s)}{1+G(s)}
$$

In [100]:
s, t = sy.symbols('s t')

def constants(G, R):
    e = sy.limit(s*R/(1+G), s, 0)
    kp = sy.limit(G, s, 0)
    kv = sy.limit(s*G, s, 0)
    ka = sy.limit(s*s*G, s, 0)
    print("e(oo) =", e)
    print("Kp =", kp)
    print("Kv =", kv)
    print("Ka =", ka)
    
    
G = 100*(s+2)*(s+6)/(s*(s+3)*(s+4))
R = sy.integrals.laplace_transform(5, t, s)[0]

constants(G, R)

R = sy.integrals.laplace_transform(5*t, t, s)[0]

constants(G, R)

e(oo) = 0
Kp = oo
Kv = 100
Ka = 0
e(oo) = 1/20
Kp = oo
Kv = 100
Ka = 0
